In [1]:
def get_ball_y_trajectory(lowest_y, highest_y, last_frame_num):
    """
    Returns a sequence of y values of a parabolic movement.

    Args:
    - lowest_y: The y position of the initial and final position.
    - highest_y: The y position when the ball has the highest position in the computer screen.
    - last_frame_num: The length of the movement.

    Returns:
    - A dictionary of y values at each frame: { 0: y, 1: y, ..., last_frame_num: y }
    """
    trajectory = {}
    for frame in range(last_frame_num+1):
        t = frame / float(last_frame_num)
        y = lowest_y + (highest_y - lowest_y) * (4 * t * (1 - t))  # parabolic equation
        trajectory[frame] = y
    return trajectory

def get_ball_x_trajectory(start_x, end_x, last_frame_num):
    # Calculate the distance the ball needs to travel
    distance = end_x - start_x

    # Calculate the increment per frame
    increment = distance / last_frame_num

    # Create a dictionary to store the x position for each frame
    positions = {}
    for frame_num in range(last_frame_num + 1):
        # Calculate the x position for this frame
        x_position = start_x + increment * frame_num

        # Store the x position in the dictionary
        positions[frame_num] = x_position

    return positions

def merge_x_y_trajectories(x_trajectory, y_trajectory):
    merged = {}
    for i in range(len(x_trajectory)):
        merged[i] = [x_trajectory[i], y_trajectory[i]]
    return merged

def get_parabola_trajectory(start_x, end_x, lowest_y, highest_y, last_frame_num):
    """
    Returns a sequence of ball positions of a parabolic movement.

    Returns:
    - A dictionary of ball positions at each frame: { 0: [x0, y0], 1: [x1, y1], ... }
    """
    x_trajectory = get_ball_x_trajectory(start_x, end_x, last_frame_num)
    y_trajectory = get_ball_y_trajectory(lowest_y, highest_y, last_frame_num)
    trajectory = merge_x_y_trajectories(x_trajectory, y_trajectory)
    return trajectory

In [2]:
def get_ball_y_trajectoryl(lowest_y, highest_y, last_frame_num):
    """
    Returns a sequence of y values of a linear bouncing movement.

    Args:
    - lowest_y: The y position of the initial position and the lower bound of the bouncing movement.
    - highest_y: The y position when the ball bounces off the upper bound and starts falling back down.
    - last_frame_num: The length of the movement.

    Returns:
    - A dictionary of y values at each frame: { 0: y, 1: y, ..., last_frame_num: y }
    """
    trajectory = {}
    apex_frame = int(last_frame_num / 2)
    for frame in range(last_frame_num+1):
        if frame <= apex_frame:
            t = frame / apex_frame
            y = lowest_y + (highest_y - lowest_y) * t
        else:
            t = (frame - apex_frame) / apex_frame
            y = highest_y - (highest_y - lowest_y) * t
        trajectory[frame] = y
    return trajectory

def get_ball_x_trajectoryl(start_x, end_x, last_frame_num):
    # Calculate the distance the ball needs to travel
    distance = end_x - start_x

    # Calculate the increment per frame
    increment = distance / last_frame_num

    # Create a dictionary to store the x position for each frame
    positions = {}
    for frame_num in range(last_frame_num + 1):
        # Calculate the x position for this frame
        x_position = start_x + increment * frame_num

        # Store the x position in the dictionary
        positions[frame_num] = x_position

    return positions

def merge_x_y_trajectories(x_trajectory, y_trajectory):
    merged = {}
    for i in range(len(x_trajectory)):
        merged[i] = [x_trajectory[i], y_trajectory[i]]
    return merged

def get_linear_trajectory(start_x, end_x, lowest_y, highest_y, last_frame_num):
    """
    Returns a sequence of ball positions of a parabolic movement.

    Returns:
    - A dictionary of ball positions at each frame: { 0: [x0, y0], 1: [x1, y1], ... }
    """
    x_trajectory = get_ball_x_trajectoryl(start_x, end_x, last_frame_num)
    y_trajectory = get_ball_y_trajectoryl(lowest_y, highest_y, last_frame_num)
    trajectory = merge_x_y_trajectories(x_trajectory, y_trajectory)
    return trajectory

In [4]:
import cv2
import numpy as np
import random
import csv

# Set the parameters for the video
video_length = 60 * 4  # seconds
fps = 25
frame_size = (640, 480)

# Create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter('video.mp4', fourcc, fps, frame_size)

# Set the initial position of the ball
ball_pos = (100, 400)  # (frame_size[0]//2, frame_size[1]//2)

# Calculate the number of frames and frames per movement
total_frames = video_length * fps
num_movements = 50
frames_per_movement = total_frames // num_movements

# Define the movement types
movement_types = ['linear', 'parabolic']
random.shuffle(movement_types)

# Create a CSV file
csv_file = open('original_coordinates.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Frame', 'Ball Position X', 'Ball Position Y'])

annotations_file = open('annotations.csv', 'w', newline='')
annotations_writer = csv.writer(annotations_file)
annotations_writer.writerow(['Frame', 'Movement Type'])


prev_movement_type = ''  # Initialize the previous movement type 

# Generate the movements
for i in range(num_movements):
    movement_type = random.choice(movement_types)

    if movement_type == 'linear':
        # Randomly choose the width and height of the movement
        width = random.randint(100, 300)  # pixels
        height = random.randint(100, 300)  # pixels

        # Keep the ball within the frame
        width = width if 0 < (ball_pos[0] + width) and (ball_pos[0] + width) < frame_size[0] else -1 * width
        height = height if 0 < (ball_pos[1] - height) else ball_pos[1]

        # Calculate the number of frames for this movement
        movement_frames = random.randint(frames_per_movement - 10, frames_per_movement + 10)

        # Calculate a trajectory
        start_x = ball_pos[0]
        end_x = ball_pos[0] + width
        lowest_y = ball_pos[1]
        highest_y = ball_pos[1] - height
        last_frame_num = movement_frames - 1
        trajectory = get_linear_trajectory(start_x, end_x, lowest_y, highest_y, last_frame_num)

        # Generate the frames for this movement
        for j in range(movement_frames):
            # Update the position of the ball
            ball_pos = (trajectory[j][0], trajectory[j][1])

            # Create a new frame
            frame = np.zeros((frame_size[1], frame_size[0], 3), dtype=np.uint8)

            # Draw the ball in the frame
            cv2.circle(frame, (int(ball_pos[0]), int(ball_pos[1])), 10, (255, 255, 255), -1)  # white

            # Write the frame to the video
            output_video.write(frame)

            # Check if the movement type changed
            if movement_type != prev_movement_type:
                # Update the movement type to "collision" for the frame where the change occurs
                csv_writer.writerow([i * frames_per_movement, ball_pos[0], ball_pos[1]])
                annotations_writer.writerow([i * frames_per_movement, 'collision'])
            else : 
                # Write the ball position and movement type to the CSV file
                csv_writer.writerow([i * frames_per_movement + j, ball_pos[0], ball_pos[1]])
                annotations_writer.writerow([i * frames_per_movement + j, movement_type])
                
            # Update the previous movement type
            prev_movement_type = movement_type
            
    elif movement_type == 'parabolic':
        # Randomly choose the width and height of the movement
        width = random.randint(100, 300)  # pixels
        height = random.randint(100, 300)  # pixels

        # Keep the ball within the frame
        width = width if 0 < (ball_pos[0] + width) and (ball_pos[0] + width) < frame_size[0] else -1 * width
        height = height if 0 < (ball_pos[1] - height) else ball_pos[1]


        # Calculate the number of frames for this movement
        movement_frames = random.randint(frames_per_movement - 10, frames_per_movement + 10)

        # Calculate a parabola trajectory
        start_x = ball_pos[0]
        end_x = ball_pos[0] + width
        lowest_y = ball_pos[1]
        highest_y = ball_pos[1] - height
        last_frame_num = movement_frames - 1
        trajectory = get_parabola_trajectory(start_x, end_x, lowest_y, highest_y, last_frame_num)

        # Generate the frames for this movement
        for j in range(movement_frames):
            # Update the position of the ball
            ball_pos = (trajectory[j][0], trajectory[j][1])

            # Create a new frame
            frame = np.zeros((frame_size[1], frame_size[0], 3), dtype=np.uint8)

            # Draw the ball in the frame
            cv2.circle(frame, (int(ball_pos[0]), int(ball_pos[1])), 10, (0, 255, 0), -1)  # green

            # Write the frame to the video
            output_video.write(frame)


            # Check if the movement type changed
            if movement_type != prev_movement_type:
                # Update the movement type to "collision" for the frame where the change occurs
                csv_writer.writerow([i * frames_per_movement, ball_pos[0], ball_pos[1]])
                annotations_writer.writerow([i * frames_per_movement, 'collision'])
            else : 
                # Write the ball position and movement type to the CSV file
                csv_writer.writerow([i * frames_per_movement + j, ball_pos[0], ball_pos[1]])
                annotations_writer.writerow([i * frames_per_movement + j, movement_type])

            # Update the previous movement type
            prev_movement_type = movement_type
    
# Release the video writer and close the video file
output_video.release()
cv2.destroyAllWindows()
csv_file.close()
annotations_file.close()


parabolic (0, 207)
127
parabolic (1, 190)
129
parabolic (2, -258)
116
parabolic (4, -261)
111
parabolic (7, -220)
121
parabolic (8, 270)
125
parabolic (9, -177)
130
parabolic (10, 124)
117
parabolic (11, -276)
112
parabolic (12, 158)
124
parabolic (13, 154)
122
parabolic (14, -294)
130
parabolic (15, 100)
117
parabolic (18, 238)
129
parabolic (19, -230)
112
parabolic (20, 232)
117
parabolic (22, 222)
114
parabolic (23, -174)
119
parabolic (24, 232)
114
parabolic (25, -226)
125
parabolic (27, -221)
127
parabolic (30, 272)
113
parabolic (32, -205)
110
parabolic (33, 287)
115
parabolic (37, -147)
112
parabolic (38, 274)
121
parabolic (39, -127)
118
parabolic (41, 130)
125
parabolic (43, 110)
119
parabolic (45, -293)
111
parabolic (46, 214)
121
parabolic (48, 253)
111
